# Data analysis with approximate state

16th October 2019

In [28]:
tic

S = dir('*.xlsx');
nfiles = length(S);

epsi = 1E-9;
xs = 5;

Pg = zeros(1,nfiles);
Hmin = zeros(1,nfiles);

etaD = 0.1;

for i = 1:1%nfiles
    S(i).name;
    data = xlsread(S(i).name);
    
    nax = [data(1:5,1:10)'];
    mu = str2num(regexp(S(i).name,'(?<=mi).+(?=.xlsx$)','once','match'))
    f = 1/(1-exp(-etaD*mu));
    %fo = 1/etaD*exp(-mu)/(1-exp(-mu)) + (1-etaD)/etaD
    
    nax = [sum(nax)*f; nax]; 
    xiax = nax./(ones(11,1)*sum(nax,1));
    
    
    p0 = exp(-mu); % we assume postselection
    p1 = exp(-mu)*mu;
    p2 = exp(-mu)*mu^2/2;

    psum = p0 + p1 + p2
    
    N = 1 + mu + mu^2/2;
    delta = round(1000*(1-exp(-mu)*N))/1000
    %delta = 2*(1-exp(-mu)*N)

    %p0 = p0/psum;
    %p1 = p1/psum;
    %p2 = p2/psum;

    d = 15;
    kets = eye(d);
    omegax = zeros(d,d,5);

    omegax(:,:,1) = p0*kets(:,1)*kets(1,:) + p1*kets(:,2)*kets(2,:) + p2*kets(:,6)*kets(6,:);
    omegax(:,:,2) = p0*kets(:,1)*kets(1,:) + p1*kets(:,3)*kets(3,:) + p2*kets(:,7)*kets(7,:);
    omegax(:,:,3) = p0*kets(:,1)*kets(1,:) + p1*kets(:,4)*kets(4,:) + p2*kets(:,8)*kets(8,:);
    omegax(:,:,4) = p0*kets(:,1)*kets(1,:) + p1*kets(:,5)*kets(5,:) + p2*kets(:,9)*kets(9,:);

    chi = 1/2*(kets(:,2) - kets(:,3) + kets(:,4) + kets(:,5));
    chi2 = (kets(:,6) + kets(:,7) + kets(:,8) + kets(:,9) +2*kets(:,15)...
            -2*kets(:,14)+2*kets(:,13)-2*kets(:,12)+2*kets(:,11)-2*kets(:,10));
    chi2 = chi2/sqrt(trace(chi2*chi2'));

    omegax(:,:,5) = p0*kets(:,1)*kets(1,:) + p1*chi*chi' + p2*chi2*chi2';
    
    for x = 1:5
    trace(omegax(:,:,x))
    end
    
    [oa,ma] = size(xiax);

    cvx_begin sdp quiet

        variable Nae(d,d,oa,oa) hermitian semidefinite
        variable qe(oa)
        variable Pax(oa,ma)
      
        maximise delta + real(sumall(conj(repmat(omegax(:,:,xs),[1,1,oa,oa])).*Nae.*...
            permute(repmat(eye(oa),[1,1,d,d]),[3,4,1,2])))
    
        for a = 1:oa-1
            for x = 1:ma
                xiax(a,x) - sqrt(log(1/epsi)/2/sum(nax(:,x))) - delta <= trace(sum(Nae(:,:,a,:),4)*omegax(:,:,x))
                trace(sum(Nae(:,:,a,:),4)*omegax(:,:,x)) <= xiax(a,x) + sqrt(log(1/epsi)/2/sum(nax(:,x))) + delta
            end
        end
    
    
        for e = 1:oa 
            sum(Nae(:,:,:,e),3) == qe(e)*eye(d)
        end
    
        sum(qe) == 1
    
    cvx_end

    Pg(i) = cvx_optval;
    Hmin(i) = max(0,-log2(cvx_optval));

end

Hmin

toc

mu =
    0.1800
psum =
    0.9992
delta =
   1.0000e-03
ans =
    0.9992
ans =
    0.9992
ans =
    0.9992
ans =
    0.9992
ans =
    0.9992
Hmin =
  Columns 1 through 7
    0.0155         0         0         0         0         0         0
  Columns 8 through 13
         0         0         0         0         0         0
Elapsed time is 3.473271 seconds.
